# Imports

In [4]:
import cv2
from cv2 import dnn
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

import numpy as np

On va créer une liste des classes et de superclasses qu'on peut detecter. l'utilité de ces listes réside dans 2 points :
- Vu que la fonction 'predict()' du model défini en bas retourne une liste de 100 probabilités, dont on tirera l'indice de la plus grande, la liste LABELS va assurer l'association du label adéquat a l'image
- SUPER_LABELS sera utilsé pour un meilleur organisation du code lors de la detection avec le HAAR Cascade

In [ ]:
#get labels :
classLabels = "beaver, dolphin, otter, seal, whale, aquarium fish, flatfish, ray, shark, trout, orchids, poppies, roses, sunflowers, tulips, containers	bottles, bowls, cans, cups, plates, apples, mushrooms, oranges, pears, sweet peppers, clock, computer keyboard, lamp, telephone, television, bed, chair, couch, table, wardrobe, bee, beetle, butterfly, caterpillar, cockroach, bear, leopard, lion, tiger, wolf, bridge, castle, house, road, skyscraper, cloud, forest, mountain, plain, sea, camel, cattle, chimpanzee, elephant, kangaroo, fox, porcupine, possum, raccoon, skunk, crab, lobster, snail, spider, worm, baby, boy, girl, man, woman, crocodile, dinosaur, lizard, snake, turtle, hamster, mouse, rabbit, shrew, squirrel, maple, oak, palm, pine, willow, bicycle, bus, motorcycle, pickup truck, train, lawn-mower, rocket, streetcar, tank, tractor"
LABELS = classLabels.split(", ")
LABELS.sort()
print("the labels list of len :", len(LABELS))
print(LABELS)

In [ ]:
#get superlabels :
SUPER_LABELS = ["aquatic mammals","fish","flowers","food containers","household electrical devices","fruit and vegetables","household furniture","large carnivores","insects","large man-made outdoor things","large natural outdoor scenes","medium-sized mammals","large omnivores and herbivores","non-insect invertebrates","reptiles","people","trees","small mammals","vehicles"]
SUPER_LABELS.sort()
print("superclass_list len = ",len(SUPER_LABELS))
print(SUPER_LABELS)

# Get the Image
Pour avoir des images homogènes, on effectuera un resize()

In [22]:
image_name = '852c2fa5e5468761c3ae8b796ca9be85_XL.jpg'
img = cv2.imread('CLASSIFICATION/test Images/'+image_name)

In [23]:
#resizing
img = cv2.resize(img,(1000,1000))

# Haar Cascade
Cette partie sera consacré la detection en utilisant le HAAR Cascade

cascade_dic est un dictionnaire contenant les Detecteurs de toutes les superclasses, selon leurs nom.
exple : cascade_dic['insect'] fera reference a un detecteur associé a la classe insect ...

In [24]:
cascade_dic = {}
cropped_img_list = []

In [25]:
for superclass in SUPER_LABELS :
    cascade_dic[superclass] = cv2.CascadeClassifier('haar/haar_cascades/'+superclass+'_cascade.xml')
    #RQ : le nom du detecteur doit obligatoirement suivre la nomeclature : superclass+'_cascade.xml'

Pour chaque detecteur, on veut sauvegarder les coordonnées du rectangle, retournées par la fonction 'detectMultiScale()'.

Ces coordonnées seront tous dans une liste : haar_list_outputs

In [ ]:
#make it grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#---------------------------------get rectangle coordinates--------------------------
haar_list_outputs = []
for superLabel in SUPER_LABELS :
    print(superLabel)
    try :
        rectangle_list = cascade_dic[superLabel].detectMultiScale(
                                image=gray, 
                                scaleFactor=1.1, 
                                minNeighbors=50, 
                                minSize=(500,500),
                                maxSize=(1000,1000)
                            )
        haar_list_outputs.append(rectangle_list)
    except:
        continue
print(haar_list_outputs)
#-------------------------------------------------------------------------------------

Et pour chaque rectangle dans 'haar_list_outputs', on coupe l'image selon le rectangle.

l'image coupée sera stocké a son tour dans une liste : 'cropped_img_list'  (définie plus haut)

In [ ]:
#--------------------------------- cropping the images -------------------------------
for cascade_output in haar_list_outputs[] :
    for (ex,ey,ew,eh) in cascade_output:
        #draw renctangles
#         cv2.rectangle(img,(ex,ey),(ex+ew,ey+eh),(0,255,0),5)   
        #cropping
        cropped_img = img[ey:ey+eh,ex:ex+ew]
        cropped_img_list.append(cropped_img)
#-------------------------------------------------------------------------------------

Partie annexe pour afficher les images dans cropped_img_list

In [ ]:
#--------------------------------- show the cropped images ---------------------------
print(len(cropped_img_list))
for im in cropped_img_list :
    #show the image
    cv2.imshow('img with rectangle',im)
    cv2.waitKey(0)
#-------------------------------------------------------------------------------------

# Image preprocessing

Les images doivent être traîtés avant d'être données comme input au classificateur. Pour cela on effectuera principalement 2 transformations :
- redimensionnement de l'image en 224 x 224
- scaling de l'image, pour avoir une matrice 3d constitué de valeurs entre 0 et 1

Ces images transformées sont a leurs tour ajoutées dans la liste 'classifier_input_images'

In [28]:
classifier_input_images = []

In [29]:
for im in cropped_img_list :
    #resizing
    im = cv2.resize(im,(224,224))     # resize image to match model's expected sizing
    im = im.reshape(1,224,224,3)      # return the image with shaping that TF wants.

    #scaling
    im = im/255
    classifier_input_images.append(im)

In [30]:
len(classifier_input_images)

1

# Classifier

### Load a classifier
Chargement du classificateur, et le résumé de sa composition

In [ ]:
model = keras.models.load_model("CLASSIFICATION/TransferLearning_EARLYSTOP_70K_with_only1Dense100_100epoch/final_save")
model.summary()

### Classification

Enfin la prédiction !

Pour chaque image dans notre input buffer 'classifier_input_images', on effectuera une prédiction, qui retrounera un vecteur de 100 probabiltés, qu'on stock dans une liste 'prediction_list'

In [31]:
prediction_list = []
for img in classifier_input_images :
    prediction  = model.predict(img)
    prediction_list.append(prediction)

1/1 [==============================] - 0s 192ms/step


In [32]:
len(prediction_list)

1

# Output Label

### maxIndexList
on définit une fonction pour retourner la liste des indices avec la proba max + on l'affiche

Cette première fonction a pour but de retourner les 5 premiers indices max de la liste passé en paramètre.

In [ ]:
def getMaxIndexList(prediction):
    maxIndexList = []
    tmp = [i for i in prediction[0]]
    for i in range(5):
        maxIndexList.append(np.argmax(tmp))
        tmp[maxIndexList[i]] = -1
    return maxIndexList

Tandis que celle-ci, a pour but d'afficher les 5 nom de classes avec la plus grande probabilité

In [33]:
def predictionFunction(prediction):
    print("associated Label with model : \n-----------------------")
    for i in getMaxIndexList(prediction):  
        print("- acc = {prediction:.3f} : {label:s}".format(prediction = prediction[0][i],label = LABELS[i]))

Cette dernière fonction se fera pour chaque prédiction dans 'prediction_list'

RQ : la correspondance img - label se fait de maniere bijective : 
        prediction_list[0] correspond a la prédiction de l'image classifier_input_images[0] etc ...

In [34]:
for prediction in prediction_list:
    predictionFunction(prediction)

associated Label with model : 
-----------------------
- acc = 1.000 : mountain
- acc = 0.997 : lobster
- acc = 0.996 : tulips
- acc = 0.987 : oranges
- acc = 0.984 : flatfish
